# Natura - Reconhecimento de Produto Nomeado

In [2]:
import unicodedata, sklearn, pandas

from tqdm import tqdm
from utils import match
from sklearn.feature_extraction.text import TfidfVectorizer

tqdm.pandas()
pandas.set_option('display.max_colwidth', None)

## Carregando Dados - Nomes Diferentes

In [3]:
df1 = pandas.read_csv('nomes_diferentes.csv', sep=',')
df1.head(3)

,ANOMES,CD_PRODUTO_ITEM,DC_PRODUTO_ITEM,DC_CATEGORIA_ITEM
0,2024-06,100144,HUMOR OFF LINE IOGURTE DESODORANTE HIDRATANTE CORPORAL PERFUMADO 125 ML,CORPO E BANHO
1,2024-11,100144,HUMOR OFF LINE IOGURTE DESODORANTE HIDRATANTE CORPORAL PERFUMADO 125 ML,CORPO E BANHO
2,2024-4,100144,HUMOR OFF LINE IOGURTE DESODORANTE HIDRATANTE CORPORAL PERFUMADO 125 ML,CORPO E BANHO


## Carregando Dados - SKUs Diferentes

In [4]:
df2 = pandas.read_csv('skus_diferentes.csv', sep=',')
df2.head(3)

,ANOMES,CD_PRODUTO_ITEM,DC_PRODUTO_ITEM,DC_CATEGORIA_ITEM
0,2024-09,103342,AMOSTRA,ESSENCIAL
1,2024-09,103347,AMOSTRA,ESSENCIAL
2,2024-11,103377,AMOSTRA,TODODIA


## Unindo Datasets

In [5]:
df = pandas.concat([df1, df2])

def remove_accents(input_str):
    if isinstance(input_str, str):
        return ''.join(
            c for c in unicodedata.normalize('NFD', input_str) if unicodedata.category(c) != 'Mn'
        )
    return input_str

df['NAME'] = (
    df['DC_PRODUTO_ITEM']
    .apply(remove_accents)
    .str.lower()
    .str.replace(r'[^a-z0-9\s]', ' ', regex=True)
    .str.replace(r'\s+', ' ', regex=True)
    .str.strip()
    .str.upper()
)

df['NAME'] = df['NAME'].astype(str)

df.head(3)

,ANOMES,CD_PRODUTO_ITEM,DC_PRODUTO_ITEM,DC_CATEGORIA_ITEM,NAME
0,2024-06,100144,HUMOR OFF LINE IOGURTE DESODORANTE HIDRATANTE CORPORAL PERFUMADO 125 ML,CORPO E BANHO,HUMOR OFF LINE IOGURTE DESODORANTE HIDRATANTE CORPORAL PERFUMADO 125 ML
1,2024-11,100144,HUMOR OFF LINE IOGURTE DESODORANTE HIDRATANTE CORPORAL PERFUMADO 125 ML,CORPO E BANHO,HUMOR OFF LINE IOGURTE DESODORANTE HIDRATANTE CORPORAL PERFUMADO 125 ML
2,2024-4,100144,HUMOR OFF LINE IOGURTE DESODORANTE HIDRATANTE CORPORAL PERFUMADO 125 ML,CORPO E BANHO,HUMOR OFF LINE IOGURTE DESODORANTE HIDRATANTE CORPORAL PERFUMADO 125 ML


In [ ]:
df

,ANOMES,CD_PRODUTO_ITEM,DC_PRODUTO_ITEM,DC_CATEGORIA_ITEM,NAME
0,2024-06,100144,HUMOR OFF LINE IOGURTE DESODORANTE HIDRATANTE CORPORAL PERFUMADO 125 ML,CORPO E BANHO,HUMOR OFF LINE IOGURTE DESODORANTE HIDRATANTE CORPORAL PERFUMADO 125 ML
1,2024-11,100144,HUMOR OFF LINE IOGURTE DESODORANTE HIDRATANTE CORPORAL PERFUMADO 125 ML,CORPO E BANHO,HUMOR OFF LINE IOGURTE DESODORANTE HIDRATANTE CORPORAL PERFUMADO 125 ML
2,2024-4,100144,HUMOR OFF LINE IOGURTE DESODORANTE HIDRATANTE CORPORAL PERFUMADO 125 ML,CORPO E BANHO,HUMOR OFF LINE IOGURTE DESODORANTE HIDRATANTE CORPORAL PERFUMADO 125 ML
3,2024-3,100144,HUMOR OFF LINE IOGURTE DESODORANTE HIDRATANTE CORPORAL PERFUMADO 125 ML,CORPO E BANHO,HUMOR OFF LINE IOGURTE DESODORANTE HIDRATANTE CORPORAL PERFUMADO 125 ML
4,2024-05,100144,HUMOR OFF LINE IOGURTE DESODORANTE HIDRATANTE CORPORAL PERFUMADO 125 ML,CORPO E BANHO,HUMOR OFF LINE IOGURTE DESODORANTE HIDRATANTE CORPORAL PERFUMADO 125 ML
...,...,...,...,...,...
2253,2023-01,112087,ÓLEO TRIFÁSICO DESODORANTE CORPORAL EKOS BURITI - 200 ML,ÓLEO,OLEO TRIFASICO DESODORANTE CORPORAL EKOS BURITI 200 ML
2254,2023-09,112087,ÓLEO TRIFÁSICO DESODORANTE CORPORAL EKOS BURITI - 200 ML,ÓLEO,OLEO TRIFASICO DESODORANTE CORPORAL EKOS BURITI 200 ML
2255,2023-10,112087,ÓLEO TRIFÁSICO DESODORANTE CORPORAL EKOS BURITI - 200 ML,ÓLEO,OLEO TRIFASICO DESODORANTE CORPORAL EKOS BURITI 200 ML
2256,2023-04,95047,ÓLEO TRIFÁSICO DESODORANTE CORPORAL EKOS BURITI - 200 ML,ÓLEO,OLEO TRIFASICO DESODORANTE CORPORAL EKOS BURITI 200 ML


: 

## Exemplo - Estudo de Caso

In [5]:
print(f'Records: {len(df)}')

df[df['DC_PRODUTO_ITEM'].isin([
    'DESODORANTE COLÔNIA HUMOR ON LINE MASCULINO 75 ML',
    'HUMOR ON-LINE DESODORANTE COLÔNIA MASCULINO - 75 ML'
])]

Records: 21918


,ANOMES,CD_PRODUTO_ITEM,DC_PRODUTO_ITEM,DC_CATEGORIA_ITEM,NAME
46,2024-06,100146,DESODORANTE COLÔNIA HUMOR ON LINE MASCULINO 75 ML,PERFUMARIA,DESODORANTE COLONIA HUMOR ON LINE MASCULINO 75 ML
47,2024-08,100146,DESODORANTE COLÔNIA HUMOR ON LINE MASCULINO 75 ML,PERFUMARIA,DESODORANTE COLONIA HUMOR ON LINE MASCULINO 75 ML
48,2024-05,100146,DESODORANTE COLÔNIA HUMOR ON LINE MASCULINO 75 ML,PERFUMARIA,DESODORANTE COLONIA HUMOR ON LINE MASCULINO 75 ML
49,2024-09,100146,DESODORANTE COLÔNIA HUMOR ON LINE MASCULINO 75 ML,PERFUMARIA,DESODORANTE COLONIA HUMOR ON LINE MASCULINO 75 ML
50,2024-10,100146,DESODORANTE COLÔNIA HUMOR ON LINE MASCULINO 75 ML,PERFUMARIA,DESODORANTE COLONIA HUMOR ON LINE MASCULINO 75 ML
51,2024-7,100146,DESODORANTE COLÔNIA HUMOR ON LINE MASCULINO 75 ML,PERFUMARIA,DESODORANTE COLONIA HUMOR ON LINE MASCULINO 75 ML
52,2024-11,100146,DESODORANTE COLÔNIA HUMOR ON LINE MASCULINO 75 ML,PERFUMARIA,DESODORANTE COLONIA HUMOR ON LINE MASCULINO 75 ML
58,2023-02,100146,HUMOR ON-LINE DESODORANTE COLÔNIA MASCULINO - 75 ML,PERFUMARIA,HUMOR ON LINE DESODORANTE COLONIA MASCULINO 75 ML
59,2023-03,100146,HUMOR ON-LINE DESODORANTE COLÔNIA MASCULINO - 75 ML,PERFUMARIA,HUMOR ON LINE DESODORANTE COLONIA MASCULINO 75 ML
60,2023-06,100146,HUMOR ON-LINE DESODORANTE COLÔNIA MASCULINO - 75 ML,PERFUMARIA,HUMOR ON LINE DESODORANTE COLONIA MASCULINO 75 ML


## Dataset Base

In [6]:
df_base = df.groupby('NAME')['NAME'].count().reset_index(name='COUNT')
df_base['ID'] = range(0, len(df_base))
df_base = df_base[['ID', 'NAME']]

print(f'Records: {len(df_base)}')
df_base.head(3)

Records: 2191


,ID,NAME
0,0,2 EM 1 BALM POS BARBA HIDRATANTE NATURA HOMEM ESSENCE 75 ML
1,1,2 EM 1 BALM POS BARBA HIDRATANTE NATURA HOMEM SAGAZ 75 ML
2,2,AGUA DE BANHO EKOS PRIPRIOCA 200 ML


## Resolvendo Nomes

In [7]:
model = TfidfVectorizer().fit(df_base['NAME'])
model

TfidfVectorizer()

In [8]:
df_base['_ID'] = df_base['NAME'].progress_apply(lambda x : match(model=model, df=df_base, _str=x, threshold=1.0))
df_base

100%|██████████| 2191/2191 [43:59<00:00,  1.20s/it]


,ID,NAME,_ID
0,0,2 EM 1 BALM POS BARBA HIDRATANTE NATURA HOMEM ESSENCE 75 ML,0
1,1,2 EM 1 BALM POS BARBA HIDRATANTE NATURA HOMEM SAGAZ 75 ML,1
2,2,AGUA DE BANHO EKOS PRIPRIOCA 200 ML,2
3,3,AGUA DE COLONIA MAMAE E BEBE FLOR DE LARANJEIRA 100 ML,3
4,4,AGUA DE COLONIA PAPAI E BEBE 100 ML,4
...,...,...,...
2186,2186,VELA PERFUMADA NATURA 774 ROSA CAPITIU 170 G,2186
2187,2187,VELA PERFUMADA NATURA 861 GUAIACO PATAQUEIRA 170 G,2187
2188,2188,VOVO DES COL MASC 100ML 2024,2188
2189,2189,VOVO DESOD COL FEM 100ML 2024,2189


In [9]:
dfm = df.merge(df_base, on='NAME', how='left')
dfm = dfm.drop('ID', axis=1).rename(columns={'_ID': 'ID'})

dfm.head(3)

,ANOMES,CD_PRODUTO_ITEM,DC_PRODUTO_ITEM,DC_CATEGORIA_ITEM,NAME,ID
0,2024-06,100144,HUMOR OFF LINE IOGURTE DESODORANTE HIDRATANTE CORPORAL PERFUMADO 125 ML,CORPO E BANHO,HUMOR OFF LINE IOGURTE DESODORANTE HIDRATANTE CORPORAL PERFUMADO 125 ML,984
1,2024-11,100144,HUMOR OFF LINE IOGURTE DESODORANTE HIDRATANTE CORPORAL PERFUMADO 125 ML,CORPO E BANHO,HUMOR OFF LINE IOGURTE DESODORANTE HIDRATANTE CORPORAL PERFUMADO 125 ML,984
2,2024-4,100144,HUMOR OFF LINE IOGURTE DESODORANTE HIDRATANTE CORPORAL PERFUMADO 125 ML,CORPO E BANHO,HUMOR OFF LINE IOGURTE DESODORANTE HIDRATANTE CORPORAL PERFUMADO 125 ML,984


In [10]:
dfm[['ANOMES', 'CD_PRODUTO_ITEM', 'DC_PRODUTO_ITEM', 'DC_CATEGORIA_ITEM', 'NAME', 'ID']][50:100]

,ANOMES,CD_PRODUTO_ITEM,DC_PRODUTO_ITEM,DC_CATEGORIA_ITEM,NAME,ID
50,2024-10,100146,DESODORANTE COLÔNIA HUMOR ON LINE MASCULINO 75 ML,PERFUMARIA,DESODORANTE COLONIA HUMOR ON LINE MASCULINO 75 ML,551
51,2024-7,100146,DESODORANTE COLÔNIA HUMOR ON LINE MASCULINO 75 ML,PERFUMARIA,DESODORANTE COLONIA HUMOR ON LINE MASCULINO 75 ML,551
52,2024-11,100146,DESODORANTE COLÔNIA HUMOR ON LINE MASCULINO 75 ML,PERFUMARIA,DESODORANTE COLONIA HUMOR ON LINE MASCULINO 75 ML,551
53,2024-04,100146,HUMOR ON LINE DESODORANTE COLÔNIA MASCULINO 75 ML,PERFUMARIA,HUMOR ON LINE DESODORANTE COLONIA MASCULINO 75 ML,551
54,2024-06,100146,HUMOR ON LINE DESODORANTE COLÔNIA MASCULINO 75 ML,PERFUMARIA,HUMOR ON LINE DESODORANTE COLONIA MASCULINO 75 ML,551
55,2024-02,100146,HUMOR ON LINE DESODORANTE COLÔNIA MASCULINO 75 ML,PERFUMARIA,HUMOR ON LINE DESODORANTE COLONIA MASCULINO 75 ML,551
56,2024-05,100146,HUMOR ON LINE DESODORANTE COLÔNIA MASCULINO 75 ML,PERFUMARIA,HUMOR ON LINE DESODORANTE COLONIA MASCULINO 75 ML,551
57,2024-03,100146,HUMOR ON LINE DESODORANTE COLÔNIA MASCULINO 75 ML,PERFUMARIA,HUMOR ON LINE DESODORANTE COLONIA MASCULINO 75 ML,551
58,2023-02,100146,HUMOR ON-LINE DESODORANTE COLÔNIA MASCULINO - 75 ML,PERFUMARIA,HUMOR ON LINE DESODORANTE COLONIA MASCULINO 75 ML,551
59,2023-03,100146,HUMOR ON-LINE DESODORANTE COLÔNIA MASCULINO - 75 ML,PERFUMARIA,HUMOR ON LINE DESODORANTE COLONIA MASCULINO 75 ML,551


In [11]:
dfm[dfm['ID'] < 0][:50]

,ANOMES,CD_PRODUTO_ITEM,DC_PRODUTO_ITEM,DC_CATEGORIA_ITEM,NAME,ID
817,2023-11,103382,REFIL CREME SORBET DESODORANTE NUTRITIVO PARA O CORPO TODODIA MANGA ROSA E ÁGUA DE COCO - 400 ML,CORPO,REFIL CREME SORBET DESODORANTE NUTRITIVO PARA O CORPO TODODIA MANGA ROSA E AGUA DE COCO 400 ML,-1
818,2023-03,103382,REFIL CREME SORBET DESODORANTE NUTRITIVO PARA O CORPO TODODIA MANGA ROSA E ÁGUA DE COCO - 400 ML,CORPO,REFIL CREME SORBET DESODORANTE NUTRITIVO PARA O CORPO TODODIA MANGA ROSA E AGUA DE COCO 400 ML,-1
819,2023-12,103382,REFIL CREME SORBET DESODORANTE NUTRITIVO PARA O CORPO TODODIA MANGA ROSA E ÁGUA DE COCO - 400 ML,CORPO,REFIL CREME SORBET DESODORANTE NUTRITIVO PARA O CORPO TODODIA MANGA ROSA E AGUA DE COCO 400 ML,-1
820,2024-02,103382,REFIL CREME SORBET DESODORANTE NUTRITIVO PARA O CORPO TODODIA MANGA ROSA E ÁGUA DE COCO - 400 ML,CORPO,REFIL CREME SORBET DESODORANTE NUTRITIVO PARA O CORPO TODODIA MANGA ROSA E AGUA DE COCO 400 ML,-1
821,2024-1,103382,REFIL CREME SORBET DESODORANTE NUTRITIVO PARA O CORPO TODODIA MANGA ROSA E ÁGUA DE COCO - 400 ML,CORPO,REFIL CREME SORBET DESODORANTE NUTRITIVO PARA O CORPO TODODIA MANGA ROSA E AGUA DE COCO 400 ML,-1
822,2023-10,103382,REFIL CREME SORBET DESODORANTE NUTRITIVO PARA O CORPO TODODIA MANGA ROSA E ÁGUA DE COCO - 400 ML,CORPO,REFIL CREME SORBET DESODORANTE NUTRITIVO PARA O CORPO TODODIA MANGA ROSA E AGUA DE COCO 400 ML,-1
823,2023-01,103382,REFIL CREME SORBET DESODORANTE NUTRITIVO PARA O CORPO TODODIA MANGA ROSA E ÁGUA DE COCO - 400 ML,CORPO,REFIL CREME SORBET DESODORANTE NUTRITIVO PARA O CORPO TODODIA MANGA ROSA E AGUA DE COCO 400 ML,-1
824,2023-08,103382,REFIL CREME SORBET DESODORANTE NUTRITIVO PARA O CORPO TODODIA MANGA ROSA E ÁGUA DE COCO - 400 ML,CORPO,REFIL CREME SORBET DESODORANTE NUTRITIVO PARA O CORPO TODODIA MANGA ROSA E AGUA DE COCO 400 ML,-1
825,2023-09,103382,REFIL CREME SORBET DESODORANTE NUTRITIVO PARA O CORPO TODODIA MANGA ROSA E ÁGUA DE COCO - 400 ML,CORPO,REFIL CREME SORBET DESODORANTE NUTRITIVO PARA O CORPO TODODIA MANGA ROSA E AGUA DE COCO 400 ML,-1
826,2023-02,103382,REFIL CREME SORBET DESODORANTE NUTRITIVO PARA O CORPO TODODIA MANGA ROSA E ÁGUA DE COCO - 400 ML,CORPO,REFIL CREME SORBET DESODORANTE NUTRITIVO PARA O CORPO TODODIA MANGA ROSA E AGUA DE COCO 400 ML,-1


In [12]:
dfm[dfm['ID'] < 0]['ID'].nunique()

1